In [204]:
#LSTM algo

In [302]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install tensorflow

In [305]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
import keras
from keras import layers

In [306]:
pd.options.display.max_columns=100

In [307]:
#To Load The data
df_train = pd.read_csv('/Users/unmeshmhatre/Desktop/Data_Analysis/24-09-2023/train.csv')
df_features = pd.read_csv('/Users/unmeshmhatre/Desktop/Data_Analysis/24-09-2023/features.csv')

In [308]:
# convert Date to datetime
df_train["Date"] = pd.to_datetime(df_train["Date"])
df_features["Date"] = pd.to_datetime(df_features["Date"])

In [309]:
df_train.shape , df_features.shape

((421570, 5), (8190, 12))

In [310]:
df_features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [311]:
df_train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [312]:
df_train.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [313]:
df_train.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


In [314]:
df_features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [315]:
train_data = pd.merge(df_train, df_features, on=['Date','Store'], how='inner')

In [316]:
train_data.tail(30)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
421540,45,41,2012-10-26,1954.67,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421541,45,42,2012-10-26,4894.78,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421542,45,44,2012-10-26,3144.09,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421543,45,46,2012-10-26,13329.25,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421544,45,52,2012-10-26,1104.16,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421545,45,54,2012-10-26,23.92,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421546,45,55,2012-10-26,3458.17,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421547,45,56,2012-10-26,937.72,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421548,45,58,2012-10-26,275.00,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421549,45,59,2012-10-26,191.98,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False


In [317]:
train_data.isna().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
dtype: int64

In [318]:
cols_to_drop = ["IsHoliday_y","MarkDown1", "MarkDown2", "MarkDown3","MarkDown4","MarkDown5","CPI","Temperature","Fuel_Price","Unemployment"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [319]:
#subset of one store
#train_data = train_data.loc[train_data["Store"] == 1]

In [320]:
train_data.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
1,1,2,2010-02-05,50605.27,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
6,1,7,2010-02-05,21084.08,False
7,1,8,2010-02-05,40129.01,False
8,1,9,2010-02-05,16930.99,False
9,1,10,2010-02-05,30721.50,False


In [321]:
train_data.shape

(421570, 5)

In [322]:
duplicates = train_data[train_data.duplicated()]

# To Display the duplicate rows from dataset
if not duplicates.empty:
    print("Duplicate Rows:")
    print(duplicates)
else:
    print("No duplicate rows found.")

# Removing the duplicate rows
df_cleaned = train_data.drop_duplicates()

No duplicate rows found.


In [323]:
#check for the outliers
time_series_column = 'Weekly_Sales'
Q1 = train_data[time_series_column].quantile(0.25)
Q3 = train_data[time_series_column].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = train_data[(train_data[time_series_column] < lower_bound) | (train_data[time_series_column] > upper_bound)]
outliers

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
1,1,2,2010-02-05,50605.27,False
36,1,38,2010-02-05,115564.35,False
37,1,40,2010-02-05,66780.63,False
56,1,72,2010-02-05,98499.12,False
65,1,90,2010-02-05,76419.47,False
...,...,...,...,...,...
421497,45,92,2012-10-19,48108.38,False
421500,45,95,2012-10-19,48434.97,False
421538,45,38,2012-10-26,47799.07,False
421564,45,92,2012-10-26,54608.75,False


In [324]:
train_data = train_data[~train_data.index.isin(outliers.index)]

In [325]:
train_data

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
...,...,...,...,...,...
421563,45,91,2012-10-26,16330.84,False
421565,45,93,2012-10-26,2487.80,False
421566,45,94,2012-10-26,5203.31,False
421568,45,97,2012-10-26,6817.48,False


In [326]:
cols_to_drop = ["Store","Dept"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [327]:
train_data

,Date,Weekly_Sales,IsHoliday_x
0,2010-02-05,24924.50,False
2,2010-02-05,13740.12,False
3,2010-02-05,39954.04,False
4,2010-02-05,32229.38,False
5,2010-02-05,5749.03,False
...,...,...,...
421563,2012-10-26,16330.84,False
421565,2012-10-26,2487.80,False
421566,2012-10-26,5203.31,False
421568,2012-10-26,6817.48,False


In [328]:
train_data.set_index('Date', inplace=True)
train_data

,Weekly_Sales,IsHoliday_x
Date,,
2010-02-05,24924.50,False
2010-02-05,13740.12,False
2010-02-05,39954.04,False
2010-02-05,32229.38,False
2010-02-05,5749.03,False
...,...,...
2012-10-26,16330.84,False
2012-10-26,2487.80,False
2012-10-26,5203.31,False


In [329]:
#resample data on weekly basis 
resampled_df = train_data.resample('W').mean()

In [330]:
resampled_df.isna().sum()

Weekly_Sales    0
IsHoliday_x     0
dtype: int64

In [331]:
resampled_df = resampled_df.dropna()

In [332]:
resampled_df

,Weekly_Sales,IsHoliday_x
Date,,
2010-02-07,10371.766994,0.0
2010-02-14,10945.257698,1.0
2010-02-21,10730.564159,0.0
2010-02-28,9917.680665,0.0
2010-03-07,10313.762879,0.0
...,...,...
2012-09-30,9994.956797,0.0
2012-10-07,10252.912027,0.0
2012-10-14,10409.566168,0.0


In [333]:
#train_size = int(len(train_data) * 0.80)
#train_size = int(len(resampled_df) * 0.80)
#train_data_1 = resampled_df.iloc[:train_size]
#test_data_1 = resampled_df.iloc[train_size:]

In [334]:
resampled_df

,Weekly_Sales,IsHoliday_x
Date,,
2010-02-07,10371.766994,0.0
2010-02-14,10945.257698,1.0
2010-02-21,10730.564159,0.0
2010-02-28,9917.680665,0.0
2010-03-07,10313.762879,0.0
...,...,...
2012-09-30,9994.956797,0.0
2012-10-07,10252.912027,0.0
2012-10-14,10409.566168,0.0


In [335]:
# Extract the sales values as the target variable
sales_data = resampled_df['Weekly_Sales'].values.reshape(-1, 1)

In [336]:
sales_data

array([[10371.76699366],
       [10945.25769797],
       [10730.56415932],
       [ 9917.6806652 ],
       [10313.7628789 ],
       [10389.78591394],
       [10640.45740822],
       [10344.50821757],
       [11097.25908546],
       [10869.52944835],
       [10316.21398302],
       [10428.15679364],
       [10277.43561039],
       [11052.3360545 ],
       [10738.87556544],
       [10656.50432513],
       [11045.26117314],
       [11241.6143064 ],
       [11008.89574548],
       [10973.69348151],
       [10965.6487472 ],
       [11082.24975113],
       [10864.76056818],
       [10874.36817875],
       [10772.27744177],
       [10783.47059193],
       [11119.32463883],
       [10984.40713053],
       [10909.21596081],
       [10826.37395716],
       [10574.34128791],
       [10484.16356352],
       [10103.68966495],
       [ 9875.57415018],
       [10003.47528247],
       [10489.95441783],
       [10226.05357248],
       [10264.88046657],
       [10452.7352926 ],
       [10520.97402081],


In [337]:
# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
sales_data = scaler.fit_transform(sales_data)

In [338]:
sales_data

array([[0.2812477 ],
       [0.40893527],
       [0.36113381],
       [0.18014548],
       [0.2683331 ],
       [0.28525961],
       [0.34107156],
       [0.27517854],
       [0.44277835],
       [0.3920744 ],
       [0.26887883],
       [0.29380288],
       [0.26024484],
       [0.43277625],
       [0.36298434],
       [0.3446444 ],
       [0.43120103],
       [0.47491901],
       [0.42310427],
       [0.4152665 ],
       [0.41347534],
       [0.43943652],
       [0.39101261],
       [0.39315174],
       [0.37042126],
       [0.37291341],
       [0.44769124],
       [0.41765189],
       [0.40091059],
       [0.38246584],
       [0.32635082],
       [0.30627277],
       [0.22156034],
       [0.17077047],
       [0.19924763],
       [0.3075621 ],
       [0.24880464],
       [0.25744944],
       [0.29927527],
       [0.31446861],
       [0.31334967],
       [0.32583019],
       [0.65971096],
       [0.39302311],
       [0.53659071],
       [0.66305806],
       [1.        ],
       [0.219

In [339]:
# To split the data in 70:30 ratio
train_size = int(0.7 * len(sales_data))
train_data_1 = sales_data[:train_size]

In [340]:
        x_train = []
        y_train = []

        for i in range(20, len(train_data_1)):
            x_train.append(train_data_1[i-20:i, 0])
            y_train.append(train_data_1[i, 0])

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        print(x_train.shape)

        test = sales_data[train_size-20:, :]
        x_test = []
        y_test = sales_data[train_size:]

        for i in range(20, len(test)):
            x_test.append(test[i-20:i, 0])

        x_test = np.array(x_test)
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
        print(sales_data.shape,x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(80, 20, 1)
(143, 1) (80, 20, 1) (80,) (43, 20, 1) (43, 1)


In [341]:
# To Build the LSTM model
model = keras.Sequential()
model.add(layers.LSTM(50, return_sequences=True, input_shape=(20, 1)))
model.add(layers.LSTM(50, return_sequences=False))
model.add(layers.Dense(15))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 20, 50)            10400     
                                                                 
 lstm_15 (LSTM)              (None, 50)                20200     
                                                                 
 dense_14 (Dense)            (None, 15)                765       
                                                                 
 dense_15 (Dense)            (None, 1)                 16        
                                                                 
Total params: 31381 (122.58 KB)
Trainable params: 31381 (122.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [342]:
# Train the LSTM model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=10, batch_size=64,verbose = 2)

Epoch 1/10
2/2 - 2s - loss: 0.1714 - 2s/epoch - 767ms/step
Epoch 2/10
2/2 - 0s - loss: 0.0867 - 22ms/epoch - 11ms/step
Epoch 3/10
2/2 - 0s - loss: 0.0397 - 21ms/epoch - 11ms/step
Epoch 4/10
2/2 - 0s - loss: 0.0269 - 27ms/epoch - 14ms/step
Epoch 5/10
2/2 - 0s - loss: 0.0339 - 24ms/epoch - 12ms/step
Epoch 6/10
2/2 - 0s - loss: 0.0344 - 24ms/epoch - 12ms/step
Epoch 7/10
2/2 - 0s - loss: 0.0289 - 24ms/epoch - 12ms/step
Epoch 8/10
2/2 - 0s - loss: 0.0248 - 21ms/epoch - 11ms/step
Epoch 9/10
2/2 - 0s - loss: 0.0243 - 25ms/epoch - 12ms/step
Epoch 10/10
2/2 - 0s - loss: 0.0243 - 25ms/epoch - 12ms/step


In [343]:
# for predicting the sales

predictions = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [344]:
predictions.shape,y_test.shape

((43, 1), (43, 1))

In [345]:
prediction_sale = predictions
y_pred = scaler.inverse_transform(prediction_sale)

In [346]:
y_test_data = y_test
y_test_data = scaler.inverse_transform(y_test_data.reshape(-1, 1))

In [347]:
 rmse_value = np.sqrt(np.mean(y_pred - y_test_data) ** 2)
 rmse_value

46.06092350237956

In [348]:
Data = pd.DataFrame()

In [349]:
y_test_1 = y_test_data.flatten()
y_pred = y_pred.flatten()

In [350]:
Data['Actual Sales'] = y_test_1
Data['Predicted Sales'] = y_pred

In [351]:
Data

,Actual Sales,Predicted Sales
0,9683.870761,10636.742188
1,9108.582694,10641.990234
2,9246.933477,10602.216797
3,9178.073087,10538.757812
4,9685.453986,10460.756836
5,10362.511024,10389.927734
6,10671.778309,10342.614258
7,10130.651289,10320.064453
8,10037.057146,10301.259766
9,10352.812416,10285.652344


In [352]:
print("Root Mean Square Error for the LSTM Model:",rmse_value)

Root Mean Square Error for the LSTM Model: 46.06092350237956
